Introduction 
============

The purpose of this document is to expose in a comprehensive way how the
spike sorting of a "large" data set can be performed with "simple" tools
built around the `Python` language. The data were recorded from a locust
*Schistocerca americana* antennal lobe (the first olfactory relay,
equivalent of the *olfactory bulb* of vertebrates). A total of 1 hour
and 40 minutes of spontaneous activity was recorded as well as responses
to 150 stimulation with citral. The set is publicly available on
[zenodo](https://zenodo.org/record/21589) (DOI:
[10.5281/zenodo.21589](http://dx.doi.org/10.5281/zenodo.21589)). The
recording setting is described in Pouzat, Mazor and Laurent (2002)---[Using noise signature to optimize spike-sorting and to assess neuronal classification quality.](http://xtof.perso.math.cnrs.fr/pdf/Pouzat+:2002.pdf) *Journal of Neuroscience Methods* **122**: 43-57---and a picture of the recording situation can be seen of the third slide
of Pouzat (2014)---Peri-Stimulus Time Histograms Estimation Through Poisson Regression Without Generalized Linear Models. Zenodo. [10.5281/zenodo.14660](http://dx.doi.org/10.5281/zenodo.14660). The purpose of these long recordings was probing
interactions between neurons and how they are modified by a stimulus.

There is no claim that the analysis presented in the sequel is "The" way
to analyze these data; it is just one *working* way. The motivation, as
a referee, is to have an explicit example to show to authors who all too
often tend to analyze their data *en bloc*. I'm advocating instead a
piecemeal approach were a first stretch of data is initially used to
build a model--that is, a catalog of waveform, one per neuron and per
recording site--while template matching is applied to the subsequent
recorded minutes using a simple trend tracking.

The following analysis was performed with the
[anaconda](https://store.continuum.io/cshop/anaconda/) distribution of
`Python 3`.

Getting the data
================

The data are stored in [HDF5](http://www.hdfgroup.org/HDF5/) format on
the [zenodo](https://zenodo.org/) server. They are all contained in a
file named `locust20010201.hdf5`. The data within this file have an
hierarchical organization similar to the one of a file system (one of
the main ideas of the HDF5 format).

The data can be downloaded and loaded as follows:

In [1]:
from urllib.request import urlretrieve
import os
import h5py
import numpy as np

name = 'locust20010201.hdf5'
distantfile = 'https://zenodo.org/record/21589/files/'+name
localfile = name
if not os.path.exists(localfile):
    urlretrieve(distantfile, localfile)
hdf = h5py.File(localfile,'r')

# read 3 trials (=3 segments)
ch_names = ['ch09','ch11','ch13','ch16']
trial_names = ['trial_01', 'trial_02', 'trial_03']
sigs_by_trials = []
for trial_name in trial_names:
    sigs = np.array([hdf['Continuous_1'][trial_name][name][...] for name in ch_names]).transpose()
    sigs = (sigs.astype('float32') - 2**15.) / 2**15
    sigs_by_trials.append(sigs)

sampling_rate = 15000.

## Exploring `HDF5` files

The `h5py` module provides an extensive access to `HDF5` files many features like the [attributes](http://www.hdfgroup.org/HDF5/doc/UG/HDF5_Users_Guide-Responsive%20HTML5/index.html#t=HDF5_Users_Guide/Attributes/HDF5_Attributes.htm). The file we just opened has for instance a _LabBook_ attribute that we can visualize with:

In [2]:
print(hdf.attrs['LabBook'])

Animal: young adult female
The data come from the second probe penetration in the right antennal lobe.
Nice activity on tetrode 9/11/13/16 with response to citral.

Continuous_1: 90 acquisitions 29 seconds long with 1 s between end and start.
Continuous_2: 20 acquisitions 29 seconds long with 1 s between end and start. 30 MICROMETERS DEEPER TO RECOVER STRONG SIGNAL.
Citral_1: 50 stimulations with pure citral (3 s before / 1 s citral / 25 s after) 1 s between end and start. AT THE END FEW DROPS OF SOLUTION AND PROBE MOVED 10 MICROMETERS DEEPER.
Citral_2: 50 stimulations with pure citral (10 s before / 1 s citral / 18 s after) 1 s between end and start.
Citral_3: 50 stimulations with pure citral (10 s before / 1 s citral / 18 s after) 1 s between end and start.
Continuous_3: 50 acquisitions 29 seconds long with 1 s between end and start.
Continuous_4: 50 acquisitions 29 seconds long with 1 s between end and start. THE FIRST 45 ACQUISITIONS ARE AVAILABLE THE LAST 5 HAVE BEEN LOST (CD CORR

We can get the names of the different [groups](http://www.hdfgroup.org/HDF5/doc/UG/HDF5_Users_Guide-Responsive%20HTML5/index.html#t=HDF5_Users_Guide/Groups/HDF5_Groups.htm) with:

In [3]:
for name in hdf:
    print(name)

Citral_1
Citral_2
Citral_3
Continuous_1
Continuous_2
Continuous_3
Continuous_4


We can get the names of the first five subgroups of group `Continuous_1` with:

In [4]:
Continuous_1_names = []
for name in hdf['Continuous_1']:
    Continuous_1_names.append(name)
print(Continuous_1_names[:5])

['trial_01', 'trial_02', 'trial_03', 'trial_04', 'trial_05']


We see that our first command loaded in RAM the first 3 elements of

In [5]:
len(Continuous_1_names)

90

trials.

The content of the `log_file_content` attribute of group `Continuous_1`
is visualized with (the first 518 characters are shown):

In [6]:
print(hdf['Continuous_1'].attrs['log_file_content'][:518])

Experiment Parameters:
  number of trials: 90
  trial length: 29 sec
  delay to odor: 3 sec
  odor duration: 1000 msec
  interval between start of trials: 30 sec
  master8 channel: 8
Continue_1 started recording: 	Thu Feb  1 16:26:11 2001
Continue_1 stopped recording: 	Thu Feb  1 16:26:40 2001
Continue_1 started recording: 	Thu Feb  1 16:26:41 2001
Continue_1 stopped recording: 	Thu Feb  1 16:27:10 2001
Continue_1 started recording: 	Thu Feb  1 16:27:11 2001
Continue_1 stopped recording: 	Thu Feb  1 16:27:40 2001


An import remark on the data
----------------------------

**The data came out of the A/D converter as 16 bit unsigned integers and were stored on the computer hard-drive as 32 bit unsigned integers** this explains the line `sigs = (sigs.astype('float32') - 2**15.) / 2**15` in the `for loop` of our first code block, where the 32 bit unsigned integers are converted to 32 bit floats with the proper offset to have zero at zero (this last maneuver is in fact superfluous since we will subsequently normalize the data). The data were moreover band-pass filtered between 300 and 5 kHz before A/D conversion and sampled at 15 kHz. 

Loading modules and code
========================

We are going to use the usual scientific python modules, [Numpy](http://docs.scipy.org/doc/numpy/), [Matplotlib](http://matplotlib.org/contents.html), [Pandas](http://pandas.pydata.org/pandas-docs/stable/). We will also use [Seaborn](http://stanford.edu/~mwaskom/software/seaborn/) and, of course, [tridesclous](https://github.com/tridesclous/tridesclous). So we start by loading all these modules:

In [7]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
import seaborn as sns

from tridesclous import SpikeSorter, SpikeSortingWindow